<a href="https://colab.research.google.com/github/Rchen89/demo-repo/blob/quick-test/COGS_Automation_Notebook_step2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="img/Adaptive-logo-long.png" width=300 align=right>

# Cost of Sales Calculation Process Automation (Step 2)

## Scope of this notebook

The objective of this notebook is to provide a Read–Eval–Print Loop (REPL) tool to support the analysis for the automation of the Cost of sales calculation process. It is important to note that this notebook doesn't replace the documentation of the analysis but rather complement it. Proving a way to quickly test our assumptions, hypothesis and experiment ideas. 

The initial section contains mainly boilerplate code. This is a temporary measure to ensure we can mount and work with files stored in Google drive. While not optimal, this approach enable us to piggyback on the built-in authentication process.

This workbook is the second step of the workbook series of Cost of Sales Calculation Process Automation. The output from the previous workbook (step 1) will be used in this workbook as the input to generate various output files


 



### *Global Utility Parameters and Functions*

#### Global Libraries

In [ ]:
## Global Libraries 
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import os


import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_theme()

#### Global Parameters


In [ ]:
reporting_month_last_day = "2021-08-31" #@param {type:"date"}
calculation_period = datetime.strptime(reporting_month_last_day, "%Y-%m-%d")
calculation_period_calendar_month = calculation_period.month
calculation_period_calendar_year = calculation_period.year

adp_data_shared_drive_folder_id = '0AJ9N1TZkcrpeUk9PVA'
department_folder_id = "1wNSJVJWCyJ-DEOaopXMgUT8BztsmyxK-"

time_entries_with_cogs_google_folder_id = "1YLRZYFi6PFXeQN_iLQEdMVgKKxxn-hr6"
cogs_reference_table_google_folder_id = "139cDdEDktC28sb45F42c4sjTPzf7u-D6"
cogs_google_output_parent_folder_id = "1WZ_3TwuNSkg341j5BQWXBMJ-O8Fk8Zee"


#### Google Authentication

In [ ]:

# Install pyDrive
!pip install -U -q PyDrive2
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

mycreds_file = 'mycreds.json'

# This function authenticate pydrive instance and save the session credential 
def authenticate_pydrive():
  gauth = GoogleAuth()
  gauth.LoadCredentialsFile(mycreds_file)
  if gauth.credentials is None:
    # Authenticate if they're not there
    auth.authenticate_user()
    gauth.credentials = GoogleCredentials.get_application_default()
  elif gauth.access_token_expired:
    # Refresh them if expired
    gauth.Refresh()
  else:
    # Initialize the saved creds
    gauth.Authorize()
  # Save the current credentials to a file
  gauth.SaveCredentialsFile(mycreds_file)
  
  drive = GoogleDrive(gauth)
  return drive

drive = authenticate_pydrive()

#### Google Drive File Handling


In [ ]:
#This function loads the most recently modified file from a google shared drive folder based on file title, file tpye and specified folder ID
def load_most_recently_modified_file_from_shared_drive_folder(file_title, file_type, parent_folder_id, retrieved_file_format, last_modified_day_offset):
  last_modified_date_from_n_days_ago = (datetime.now() - timedelta(days=last_modified_day_offset)).strftime("%Y-%m-%dT%H:%M:%S")
  if file_title == "":
    file_list = drive.ListFile({'q': f"mimeType = '{file_type}' and parents in '{parent_folder_id}' and trashed=false and modifiedDate > '{last_modified_date_from_n_days_ago}'", 'corpora': 'teamDrive', 'teamDriveId': adp_data_shared_drive_folder_id, 'includeItemsFromAllDrives': True, 'supportsAllDrives': True}).GetList()
  else:
    file_list = drive.ListFile({'q': f"title contains '{file_title}' and mimeType = '{file_type}' and parents in '{parent_folder_id}' and trashed=false and modifiedDate > '{last_modified_date_from_n_days_ago}'", 'corpora': 'teamDrive', 'teamDriveId': adp_data_shared_drive_folder_id, 'includeItemsFromAllDrives': True, 'supportsAllDrives': True}).GetList()
  file_list.sort(key = lambda x : x["modifiedDate"], reverse = True)

  print("File list: ")
  for file in file_list:
    print(file["title"] + " | " + file["id"] + " | " + file["modifiedDate"] + " | " + file["mimeType"])

  if(len(file_list)>0):
    file_title = file_list[0]["title"]
    file_id = file_list[0]["id"]
    data = read_csv_from_drive(file_id,file_title)
    print("The File loaded: " + file_title)
    if retrieved_file_format == "file_name":
      return file_title
    elif retrieved_file_format == "file_id":
      return file_id
    elif retrieved_file_format == "data":
      return data
  else:
    raise NameError('No file has been found based on the current search criteria ')

#This function loads the most recently modified file from google my drive folder based on file title, file tpye and specified folder ID
def load_most_recently_modified_file_from_my_drive_folder(file_title, file_type, parent_folder_id, retrieved_file_format, last_modified_day_offset):
  last_modified_date_from_n_days_ago = (datetime.now() - timedelta(days=last_modified_day_offset)).strftime("%Y-%m-%dT%H:%M:%S")
  if file_title == "":
    file_list = drive.ListFile({'q': f"mimeType = '{file_type}' and parents in '{parent_folder_id}' and trashed=false and modifiedDate > '{last_modified_date_from_n_days_ago}'" }).GetList()
  else:
    file_list = drive.ListFile({'q': f"title contains '{file_title}' and mimeType = '{file_type}' and parents in '{parent_folder_id}' and trashed=false and modifiedDate > '{last_modified_date_from_n_days_ago}'" }).GetList()
  file_list.sort(key = lambda x : x["modifiedDate"], reverse = True)

  print("File list: ")
  for file in file_list:
    print(file["title"] + " | " + file["id"] + " | " + file["modifiedDate"] + " | " + file["mimeType"])

  if(len(file_list)>0):
    file_title = file_list[0]["title"]
    file_id = file_list[0]["id"]
    data = read_csv_from_drive(file_id,file_title)
    print("The File loaded: " + file_title)
    if retrieved_file_format == "file_name":
      return file_title
    elif retrieved_file_format == "file_id":
      return file_id
    elif retrieved_file_format == "data":
      return data
  else:
    raise NameError('No file has been found based on the current search criteria ') 

#This function loads the most recently modified folder from a parent google drive folder based on the specified parent folder ID
def load_most_recently_modified_folder_from_specified_folder(parent_folder_id, last_modified_day_offset):
  last_modified_date_from_n_days_ago = (datetime.now() - timedelta(days=last_modified_day_offset)).strftime("%Y-%m-%dT%H:%M:%S")
  folder_list = drive.ListFile({'q': f"mimeType = '{'application/vnd.google-apps.folder'}' and parents in '{parent_folder_id}' and trashed=false and modifiedDate > '{last_modified_date_from_n_days_ago}'" }).GetList()
  folder_list.sort(key = lambda x : x["modifiedDate"], reverse = True)

  print("Folder list: ")
  for folder in folder_list:
    print(folder["title"] + " | " + folder["id"] + " | " + folder["modifiedDate"] + " | " + folder["mimeType"])

  if(len(folder)>0):
    folder_title = folder_list[0]["title"]
    folder_id = folder_list[0]["id"]
    print("The File loaded: " + folder_id)
    return folder_id
  else:
    raise NameError('No folder has been found based on the current search criteria ')

#This function read the csv file based on specified file id from the google drive 
def read_csv_from_drive(file_id, file_name):
    dl = drive.CreateFile({'id': file_id})
    dl.GetContentFile(file_name)
    return pd.read_csv(file_name)

#This function read the csv file based on specified file id from the google drive 
def upload_file_to_google_drive(output_file_name, file_mime_type, google_drive_folder_id, temp_filename):
    file_upload = drive.CreateFile({"title":output_file_name,
                                    "mimeType": file_mime_type, 
                                    "parents" : [{"id" : google_drive_folder_id}]})
    file_upload.SetContentFile(temp_filename)
    file_upload.Upload()
    print('Uploaded file with ID {}'.format(file_upload.get('id')))

# This function converts dataframe into csv file and upload it to specified folder in google drive
def upload_file_to_drive_csv_format(data, google_drive_folder_id, output_file_name, index_included):
    temp_filename = "data.csv"
    if index_included == True:
      data.to_csv(temp_filename,index=True)
    else:
      data.to_csv(temp_filename,index=False)
    google_drive_folder_id = google_drive_folder_id
    output_file_name = output_file_name
    upload_file_to_google_drive(output_file_name, "text/csv", google_drive_folder_id, temp_filename)



#### Data Cleasing functions


In [ ]:
# Set the data type to mulitiple columns of a dataframe
def set_column_data_type(data,col_list,data_type):
    data_copy = data.copy()
    data_copy[col_list] = data_copy[col_list].astype(data_type)
    return data_copy

#### Reference Data


##### Xero Department Code Mappings

> Indented block




In [ ]:
xero_department_code_mapping = load_most_recently_modified_file_from_shared_drive_folder("departments",'text/csv',department_folder_id, "data", 365)
xero_department_code_mapping.set_index("department_name", inplace = True)

#### Load time entries with calculated costs dataframe

###### Find the folder which the most recently processed time entry file is stored 

In [ ]:
most_recent_cleansed_time_entry_file_folder_id = load_most_recently_modified_folder_from_specified_folder(cogs_google_output_parent_folder_id, 365)

##### Helper Functions


In [ ]:
# This function retrieve the time stamp from the processed time entry file from step 1 script
def derive_output_time_stamp(file_name):
  timestamp_splitter = "|"
  index = file_name.find(timestamp_splitter)
  return file_name[index+1:len(file_name)-4]

# This function derives the department code for AI template
def derive_department_code_for_AI_template(row):
  business_unit_mapping_for_AI = {
      "Adaptive Canada" : "MTL",
      "Adaptive UK" : "LDN",
      "Adaptive US" : "NY",
      "Adaptive Spain" : "BCN"
  }
  resource_BU = row["resource_BU"]
  adp_department = row["adp_department"]
  
  return adp_department + " - " + business_unit_mapping_for_AI[resource_BU]

# This function derives the reference column in the output file 
def derive_reference_column(row):
  resourced_activity = row["resourced_activity"]
  delivery_element_reference = row["delivery_element_reference"]
  if delivery_element_reference != "n/a":
    return delivery_element_reference
  else:
    return resourced_activity

# This function derives the tax rate column on the journal entry template
def derive_journal_tax_rate(row):
  return "Tax Exempt"

# This function filters the full journal entry dataframe by each business unit, then upload the resulting dataframe as csv file to target google drive folder
def process_cogs_journal_entry_by_BU(df_journal_entris_all, business_unit, output_batch_code):
  cogs_journal_template_final_BU = cogs_journal_template_final[cogs_journal_template_final["Entity"] == business_unit]
  cogs_journal_template_file_name = "COGS - Journal Template Output_" + business_unit + " | " + output_time_stamp + ".csv"
  upload_file_to_drive_csv_format(cogs_journal_template_final_BU, most_recent_cleansed_time_entry_file_folder_id, cogs_journal_template_file_name, False)
  return cogs_journal_template_final_BU
    
# This function derives the line description for B/S reconciliation file    
def derive_bs_acc_line_description(row):
  entry_units_by_day = row["entry_units_by_day"]
  daily_rate = row["bamboohr_cost_rate_daily"]
  resource = row["resource"]

  return str(calculation_period_calendar_month) + "-" + str(calculation_period_calendar_year) + " " + resource + " " + str(round(entry_units_by_day, 3)) + " days at " + str(daily_rate)

#### Load the most recently processed time entry file with calculated costs

In [ ]:
time_entries_with_calculated_cost_file_name = load_most_recently_modified_file_from_my_drive_folder('COGS_Output_2_Kimble_Time_Entries', 'text/csv', most_recent_cleansed_time_entry_file_folder_id, "file_name", 30)
output_time_stamp =  derive_output_time_stamp(time_entries_with_calculated_cost_file_name)
time_entries_with_calculated_cost = load_most_recently_modified_file_from_my_drive_folder('COGS_Output_2_Kimble_Time_Entries', 'text/csv', most_recent_cleansed_time_entry_file_folder_id, "data", 30)
time_entries_with_calculated_cost.loc[time_entries_with_calculated_cost["cost_GL"].notnull(), "cost_GL"] = time_entries_with_calculated_cost.loc[time_entries_with_calculated_cost["cost_GL"].notnull(), "cost_GL"].astype(int).astype(str)
time_entries_with_calculated_cost.loc[time_entries_with_calculated_cost["contra_GL"].notnull(), "contra_GL"] = time_entries_with_calculated_cost.loc[time_entries_with_calculated_cost["contra_GL"].notnull(), "contra_GL"].astype(int).astype(str)
time_entries_with_calculated_cost["reference"] = time_entries_with_calculated_cost.apply(lambda row: derive_reference_column(row), axis=1)
time_entries_with_calculated_cost.fillna("n/a",inplace = True)
time_entries_with_calculated_cost.head()

#### Generate AI Upload Template 


In [ ]:
time_entries_pivot_table_for_AI_template = pd.pivot_table(time_entries_with_calculated_cost, index=["resource_BU", "resource", "resource_id", "resource_type", "cost_GL", "resourced_activity", "adp_department", "delivery_element_reference", "adp_revenue_cost_category"], values = ["cost_for_activity", "entry_units_by_day"], aggfunc="sum")

time_entries_pivot_table_for_AI_template

In [ ]:
time_entries_pivot_table_flattened_for_AI_template = pd.DataFrame(time_entries_pivot_table_for_AI_template.to_records())
time_entries_pivot_table_flattened_for_AI_template["department_code_AI"] = time_entries_pivot_table_flattened_for_AI_template.apply(lambda row: derive_department_code_for_AI_template(row), axis=1)
time_entries_pivot_table_flattened_for_AI_template["reference"] = time_entries_pivot_table_flattened_for_AI_template.apply(lambda row: derive_reference_column(row), axis=1)
time_entries_pivot_table_flattened_for_AI_template = time_entries_pivot_table_flattened_for_AI_template[time_entries_pivot_table_flattened_for_AI_template["cost_GL"].str.startswith("5")]
column_order = ["department_code_AI", "resource", "resource_id", "resource_type", "cost_for_activity", "entry_units_by_day", "reference", "adp_revenue_cost_category", "adp_department"]
time_entries_pivot_table_flattened_for_AI_template = time_entries_pivot_table_flattened_for_AI_template[column_order]
time_entries_pivot_table_flattened_for_AI_template

cogs_AI_template_file_name = "COGS - AI Template Output" + " | " + output_time_stamp + ".csv"
upload_file_to_drive_csv_format(time_entries_pivot_table_flattened_for_AI_template, most_recent_cleansed_time_entry_file_folder_id, cogs_AI_template_file_name, False)

In [ ]:
time_entries_pivot_table_flattened_for_AI_template_by_days = time_entries_pivot_table_flattened_for_AI_template
column_order_by_days =  ["department_code_AI", "resource", "resource_id", "resource_type", "reference", "adp_department", "entry_units_by_day"]
time_entries_pivot_table_flattened_for_AI_template_by_days = time_entries_pivot_table_flattened_for_AI_template_by_days[column_order_by_days]

cogs_by_days_AI_template_file_name = "COGS - AI Template Output - by days" + " | " + output_time_stamp + ".csv"
upload_file_to_drive_csv_format(time_entries_pivot_table_flattened_for_AI_template_by_days, most_recent_cleansed_time_entry_file_folder_id, cogs_by_days_AI_template_file_name, False)


time_entries_pivot_table_flattened_for_AI_template_by_cost = time_entries_pivot_table_flattened_for_AI_template
column_order_by_costs = ["department_code_AI", "resource", "resource_id", "reference", "resource_type", "adp_department", "cost_for_activity"]
time_entries_pivot_table_flattened_for_AI_template_by_cost = time_entries_pivot_table_flattened_for_AI_template_by_cost[column_order_by_costs]

cogs_by_costs_AI_template_file_name = "COGS - AI Template Output - by costs" + " | " + output_time_stamp + ".csv"
upload_file_to_drive_csv_format(time_entries_pivot_table_flattened_for_AI_template_by_cost, most_recent_cleansed_time_entry_file_folder_id, cogs_by_costs_AI_template_file_name, False)


#### Generate Journal Entry Upload Template 

In [ ]:
time_entries_pivot_table_for_journal_template = pd.pivot_table(time_entries_with_calculated_cost, index=["resource_BU", "cost_GL", "adp_department", "contra_GL", "resource_department", "resourced_activity", "delivery_element_reference"], values = ["cost_for_activity"], aggfunc="sum")
time_entries_pivot_table_for_journal_template

In [ ]:
# time_entries_pivot_table_flattened_for_journal_template = pd.DataFrame(time_entries_pivot_table_for_journal_template.to_records())
# upload_file_to_drive_csv_format(time_entries_pivot_table_flattened_for_journal_template, most_recent_cleansed_time_entry_file_folder_id, "test_journal_pivot_data.csv", False)

In [ ]:
time_entries_pivot_table_flattened_for_journal_template = pd.DataFrame(time_entries_pivot_table_for_journal_template.to_records())
time_entries_pivot_table_flattened_for_journal_template = time_entries_pivot_table_flattened_for_journal_template[time_entries_pivot_table_flattened_for_journal_template["cost_GL"] != "n/a"]
time_entries_pivot_table_flattened_for_journal_template["cost_for_activity"] = time_entries_pivot_table_flattened_for_journal_template["cost_for_activity"].round(2) 
time_entries_pivot_table_flattened_for_journal_template["Entity"] = time_entries_pivot_table_flattened_for_journal_template["resource_BU"]
time_entries_pivot_table_flattened_for_journal_template["Date"] = reporting_month_last_day
time_entries_pivot_table_flattened_for_journal_template["Description"] = "COGS " + str(calculation_period_calendar_month) + "-" + str(calculation_period_calendar_year)
time_entries_pivot_table_flattened_for_journal_template["Narration"] = "COGS " + str(calculation_period_calendar_month) + "-" + str(calculation_period_calendar_year) + " - " + "See Cogs files for back up"
time_entries_pivot_table_flattened_for_journal_template["Tax Rate"] = time_entries_pivot_table_flattened_for_journal_template.apply(lambda row: derive_journal_tax_rate(row), axis = 1)
time_entries_pivot_table_flattened_for_journal_template["TrackingOption1"] = time_entries_pivot_table_flattened_for_journal_template["adp_department"].map(xero_department_code_mapping["xero_department_name"])
time_entries_pivot_table_flattened_for_journal_template["TrackingOption2"] = time_entries_pivot_table_flattened_for_journal_template.apply(lambda row: derive_reference_column(row), axis=1)
time_entries_pivot_table_flattened_for_journal_template

In [ ]:
cogs_journal_template_debit_lines = time_entries_pivot_table_flattened_for_journal_template.copy()
cogs_journal_template_debit_lines["Accounting Code"] = cogs_journal_template_debit_lines["cost_GL"]
cogs_journal_template_debit_lines["Amount"] = cogs_journal_template_debit_lines["cost_for_activity"]
journal_columns = ["Entity", "Date", "Description", "Narration", "Accounting Code", "Tax Rate", "Amount", "TrackingOption1", "TrackingOption2"]
cogs_journal_template_debit_lines = cogs_journal_template_debit_lines[journal_columns]
cogs_journal_template_debit_lines

In [ ]:
cogs_journal_template_credit_lines = time_entries_pivot_table_flattened_for_journal_template.copy()
cogs_journal_template_credit_lines["Accounting Code"] = cogs_journal_template_credit_lines["contra_GL"]
cogs_journal_template_credit_lines["Amount"] = cogs_journal_template_credit_lines["cost_for_activity"] * -1
journal_columns = ["Entity", "Date", "Description", "Narration", "Accounting Code", "Tax Rate", "Amount", "TrackingOption1", "TrackingOption2"]
cogs_journal_template_credit_lines = cogs_journal_template_credit_lines[journal_columns]
cogs_journal_template_credit_lines

In [ ]:
cogs_journal_template_final = pd.concat([cogs_journal_template_debit_lines, cogs_journal_template_credit_lines],axis = 0)
cogs_journal_template_final

In [ ]:
cogs_journal_template_final_uk = process_cogs_journal_entry_by_BU(cogs_journal_template_final, "Adaptive UK", output_time_stamp)
cogs_journal_template_final_us = process_cogs_journal_entry_by_BU(cogs_journal_template_final, "Adaptive US", output_time_stamp)
cogs_journal_template_final_spain = process_cogs_journal_entry_by_BU(cogs_journal_template_final, "Adaptive Spain", output_time_stamp)
cogs_journal_template_final_canada = process_cogs_journal_entry_by_BU(cogs_journal_template_final, "Adaptive Canada", output_time_stamp)

##### Contractor BS rec acc LC

In [ ]:
time_entries_with_calculated_cost_contractors = time_entries_with_calculated_cost[time_entries_with_calculated_cost["resource_type"] == "Contractor"]
time_entries_with_calculated_cost_contractors

In [ ]:
time_entries_pivot_for_bs_acc_contractor = pd.pivot_table(time_entries_with_calculated_cost_contractors, index=["resource_BU", "contra_GL", "resource", "bamboohr_cost_rate_daily", "delivery_element_shortname", "delivery_element_reference", "adp_department"], values = ["entry_units_by_day", "cost_for_activity"], aggfunc="sum")
time_entries_pivot_for_bs_acc_contractor

In [ ]:
time_entries_pivot_flatterned_for_bs_acc_contractor = pd.DataFrame(time_entries_pivot_for_bs_acc_contractor.to_records())
time_entries_pivot_flatterned_for_bs_acc_contractor["description"] = time_entries_pivot_flatterned_for_bs_acc_contractor.apply(lambda row: derive_bs_acc_line_description(row), axis = 1)
time_entries_pivot_flatterned_for_bs_acc_contractor["cost_on_bs"] = time_entries_pivot_flatterned_for_bs_acc_contractor["cost_for_activity"] * -1
time_entries_pivot_flatterned_for_bs_acc_contractor = time_entries_pivot_flatterned_for_bs_acc_contractor[time_entries_pivot_flatterned_for_bs_acc_contractor["cost_on_bs"] != 0]
ordered_columns = ["resource_BU", "resource", "bamboohr_cost_rate_daily", "entry_units_by_day", "cost_for_activity", "cost_on_bs", "description", "delivery_element_reference", "delivery_element_shortname", "adp_department"]
time_entries_pivot_flatterned_for_bs_acc_contractor = time_entries_pivot_flatterned_for_bs_acc_contractor[ordered_columns]
time_entries_pivot_flatterned_for_bs_acc_contractor


In [ ]:
cogs_bs_acc_file_name = "COGS - B/S acc Output" + " | " + output_time_stamp + ".csv"
upload_file_to_drive_csv_format(time_entries_pivot_flatterned_for_bs_acc_contractor, most_recent_cleansed_time_entry_file_folder_id, cogs_bs_acc_file_name, False)

This is a new line ok
